# Data Cleaning & Dataset Preparation

This notebook applies validated business rules to clean the retail dataset.
Invalid records are removed, sales and returns are separated, and clean,
analysis-ready datasets are created and stored in the `data/processed/` directory.


In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("online_retail.csv")

In [3]:
df.shape

(541909, 8)

In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  str    
 1   StockCode    541909 non-null  str    
 2   Description  540455 non-null  str    
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  str    
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  str    
dtypes: float64(2), int64(1), str(5)
memory usage: 33.1 MB


In [5]:
df=df[df['UnitPrice']>=0]

In [6]:
df.shape


(541907, 8)

## Step 2: Create Returns Dataset

- Transactions with negative Quantity or cancelled InvoiceNo are classified as returns
- Returns are separated to avoid distortion of sales and revenue analysis


In [7]:
returns_df=df[(df['Quantity']<0) | (df['InvoiceNo'].str.startswith('C'))]

In [8]:
returns_df.shape

(10624, 8)

In [9]:
returns_df['Quantity'].describe()


count    10624.000000
mean       -45.607210
std       1092.214216
min     -80995.000000
25%        -10.000000
50%         -2.000000
75%         -1.000000
max         -1.000000
Name: Quantity, dtype: float64

## Step 2B: Create Sales Dataset

- Sales transactions defined as positive Quantity, valid UnitPrice, and non-cancelled invoices
- Returns and invalid pricing rows are excluded to ensure accurate revenue analysis


In [10]:
sales_df=df[(df['Quantity']>0)&(~df['InvoiceNo'].str.startswith('C'))&(df['UnitPrice']>0)]

In [11]:
sales_df.shape

(530104, 8)

In [12]:
sales_df['Quantity'].describe()

count    530104.000000
mean         10.542037
std         155.524124
min           1.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       80995.000000
Name: Quantity, dtype: float64

In [13]:
sales_df['UnitPrice'].describe()

count    530104.000000
mean          3.907625
std          35.915681
min           0.001000
25%           1.250000
50%           2.080000
75%           4.130000
max       13541.330000
Name: UnitPrice, dtype: float64

In [14]:
sales_df['InvoiceNo'].str.startswith('C').sum()

np.int64(0)

## Step 3: Save Cleaned Data

- Final validation confirms no overlap between sales and returns datasets
- Cleaned datasets saved for downstream analysis and reproducibility


In [15]:
overlap = sales_df.index.intersection(returns_df.index)
len(overlap)


0

In [16]:
import os
os.listdir("../data")

['processed']

In [17]:
os.makedirs("../data/processed", exist_ok=True)


In [18]:
sales_df.to_csv("../data/processed/sales_cleaned.csv", index=False)
returns_df.to_csv("../data/processed/returns_cleaned.csv", index=False)


In [19]:
os.listdir("../data/processed")


['returns_cleaned.csv', 'sales_cleaned.csv']

## Data Storage and Organization

To ensure a clean, reproducible, and professional workflow, the project follows a clear
data separation strategy.

- A dedicated `data/processed/` directory was created to store cleaned and analysis-ready datasets
- Raw data is preserved in `data/raw/` and is never overwritten
- All data cleaning logic is performed in a separate notebook, and the final outputs are saved as CSV files

### Processed Files Created
- `sales_cleaned.csv` → contains validated sales transactions only
- `returns_cleaned.csv` → contains return and cancellation transactions

This approach ensures:
- Reproducibility of analysis
- Clear separation between raw and cleaned data
- Safe reuse of cleaned datasets across multiple notebooks (EDA, visualization, modeling)
